In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import matplotlib.pyplot as plt
from matplotlib import colors


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from pathlib import Path

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

training_tasks = sorted(os.listdir(training_path))


In [ ]:
# ======================================================================================
# USING DECISIONS TREES
# ======================================================================================

# IMPORT LIBRARIES AND DATASETS
import numpy as np
import pandas as pd

import os
import json

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from lightgbm import LGBMClassifier
import pdb

from pathlib import Path

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

training_tasks = sorted(os.listdir(test_path))

sample_sub = pd.read_csv(data_path/'sample_submission.csv')
sample_sub = sample_sub.set_index('output_id')
sample_sub.head()

In [ ]:
# PLOTING FUNCTION

def plot_result(test_input, test_prediction,
                input_shape):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 2, figsize=(15,15))
    test_input = test_input.reshape(input_shape[0],input_shape[1])
    axs[0].imshow(test_input, cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Actual Target')
    test_prediction = test_prediction.reshape(input_shape[0],input_shape[1])
    axs[1].imshow(test_prediction, cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Model Prediction')
    plt.tight_layout()
    plt.show()
    
def plot_test(test_prediction, task_name):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 1, figsize=(15,15))
    axs.imshow(test_prediction, cmap=cmap, norm=norm)
    axs.axis('off')
    axs.set_title(f'Test Prediction {task_name}')
    plt.tight_layout()
    plt.show()

In [ ]:
# FOR FLATTENING 2D NUMPY ARRAYS

# https://www.kaggle.com/inversion/abstraction-and-reasoning-starter-notebook
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

In [ ]:
# EXTRACT NEIGHBOURHOOD FEATURES

def get_moore_neighbours(color, cur_row, cur_col, nrows, ncols):

    if cur_row<=0: top = -1
    else: top = color[cur_row-1][cur_col]
        
    if cur_row>=nrows-1: bottom = -1
    else: bottom = color[cur_row+1][cur_col]
        
    if cur_col<=0: left = -1
    else: left = color[cur_row][cur_col-1]
        
    if cur_col>=ncols-1: right = -1
    else: right = color[cur_row][cur_col+1]
        
    return top, bottom, left, right

def get_tl_tr(color, cur_row, cur_col, nrows, ncols):
        
    if cur_row==0:
        top_left = -1
        top_right = -1
    else:
        if cur_col==0: top_left=-1
        else: top_left = color[cur_row-1][cur_col-1]
        if cur_col==ncols-1: top_right=-1
        else: top_right = color[cur_row-1][cur_col+1]   
        
    return top_left, top_right

In [ ]:
# FEATURES FOR EACH TRAIN SAMPLE

def features(task, mode = 'train'):
    current_index = 0
    number_train_pairs = len(task[mode])
    
    # Inputs = all  the values of all matrices of the training images
    total_inputs = sum([len(task[mode][i]['input'])*len(task[mode][i]['input'][0]) for i in range(number_train_pairs)])
    
    # Feature
    feature = np.zeros((total_inputs, number_features))
    
    #Output
    target = np.zeros((total_inputs,), dtype = np.int) #we don't specify the dimensions, but by default=1
    
    # Global is used to create global variables from a non-global scope i.e inside a function.
    # Global keyword is used inside a function only when we want to do assignments or when we want to change a variable
    global local_neighbours
    
    for task_number in range(number_train_pairs):
        # Input coloured image
        input_color = np.array(task[mode][task_number]['input'])
        
        # Target coloured image
        target_color = task[mode][task_number]['output']
    
        # Number of rows, columns of the input image
        input_rows, input_columns = len(task[mode][task_number]['input']), len(task[mode][task_number]['input'][0])
        
        # Number of rows, columns of the output image
        target_rows, target_columns = len(task[mode][task_number]['output']), len(task[mode][task_number]['output'][0])
        
        if (target_rows != input_rows) or (target_columns != input_columns):
            print('Number of input rows:',input_rows,'cols:',input_columns)
            print('Number of target rows:',target_rows,'cols:',target_columns)
            not_valid=1
            return None, None, 1
        
        for i in range (input_rows):
            for j in range(input_columns):
                feature[current_index, 0] = i
                feature[current_index,1] = j
                feature[current_index,2] = input_color[i][j]
                feature[current_index,3:7] = get_moore_neighbours(input_color, i, j, input_rows, input_columns)
                feature[current_index,7:9] = get_tl_tr(input_color, i, j, input_rows, input_columns)
                feature[current_index,9] = len(np.unique(input_color[i,:]))
                feature[current_index,10] = len(np.unique(input_color[:,j]))
                feature[current_index,11] = (i+j)
                feature[current_index,12] = len(np.unique(input_color[i-local_neighbours:i+local_neighbours,
                                                             j-local_neighbours:j+local_neighbours]))
        
                target[current_index] = target_color[i][j]
                current_index += 1
        
        
    return feature, target, 0

In [ ]:
## TRAINING AND PREDICTION

all_task_ids = sorted(os.listdir(test_path))# test dataset

number_features = 13
local_neighbours = 3
valid_scores = {}
for task_id in all_task_ids:

    task_file = str(test_path / task_id)
    with open(task_file, 'r') as f:
        task = json.load(f)
        
    feature, target,not_valid = features(task)
    if not_valid:
        print('ignoring task', task_file)
        print()
        not_valid = 0
        continue
        
        
    # we use the last train example for validation
    input_rows, input_columns = len(task['train'][-1]['input']), len(task['train'][-1]['input'][0]) # index -1 stands for the last training pair
    validation_index = len(feature) - input_rows*input_columns
    
    train_features = feature[:validation_index]
    validation_features = feature[validation_index:, :]

    train_target = target[:validation_index]
    validation_target = target[validation_index:]
    
    
    #     check if validation set has a new color  --> with set() -> if a = {1,1,3,4,5,5} --> set(a) = {1,3,4,5}
    #     if so make the mapping color independant
    if len(set(validation_target) - set(train_target)):
        print('set(val_target)', set(validation_target))
        print('set(train_target)', set(train_target))
        print('Number of colors are not same')
        print('cant handle new colors. skipping')
        continue

    # LIGHT GBM ALGORITHM:
    # https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
    # Light GBM grows tree vertically while other algorithm grows trees horizontally meaning that Light GBM grows tree leaf-wise while other algorithm grows level-wise.
    lgb = LGBMClassifier(n_estimators=50, n_jobs=-1) #initialise
    lgb.fit(feature, target, verbose=-1)
    
    #training in input pairs is done
    
    
    #---------------------------------------------------------
    #TESTING
    num_test_pairs = len(task['test'])
    for task_num in range(num_test_pairs):
        current_index = 0
        input_color = np.array(task['test'][task_num]['input'])
        input_rows, input_columns = len(task['test'][task_num]['input']), len(task['test'][task_num]['input'][0])
        feature = np.zeros((input_rows*input_columns, number_features))
        unique_col = {column: i for i, column in enumerate(sorted(np.unique(input_color)))} # identify the colours that appear and enumerate them 1, 2,...
        
        for i in range(input_rows):
            for j in range(input_columns):
                feature[current_index, 0] = i
                feature[current_index, 1] = j
                feature[current_index, 2] = input_color[i][j]
                feature[current_index, 3:7] = get_moore_neighbours(input_color, i, j, input_rows, input_columns)
                feature[current_index, 7:9] = get_tl_tr(input_color, i, j, input_rows, input_columns)
                feature[current_index, 9] = len(np.unique(input_color[i, :]))
                feature[current_index, 10] = len(np.unique(input_color[:, j]))
                feature[current_index, 11] = (i+j)
                feature[current_index, 12] = len(np.unique(input_color[i-local_neighbours:i+local_neighbours,j-local_neighbours:j+local_neighbours]))
                current_index += 1
                
        print('Made predictions for ', task_id[:-5])
        preds = lgb.predict(feature).reshape(input_rows, input_columns)
        preds = preds.astype(int).tolist()
        plot_test(preds, task_id)
        sample_sub.loc[f'{task_id[:-5]}_{task_num}', 'output'] = flattener(preds)
    
    
    
    
    
    

In [ ]:
sample_sub.head()

In [ ]:
sample_sub.to_csv('submission.csv')